In [ ]:
import os
import time
import shutil
import math
import numpy as np
import ansys.speos.script as script
import ansys.speos.core as core
from ansys.api.speos.sensor.v1 import camera_sensor_pb2


tests_data_path = "/app/assets"


def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

def rotation_matrix(theta, phi, coordinate):
    theta_rad = theta* math.pi/180
    phi_rad = phi*math.pi/180
    rotation_phi = np.array([
        [math.cos(phi_rad), -math.sin(phi_rad), 0],
        [math.sin(phi_rad), math.cos(phi_rad), 0],
        [0, 0, 1]
    ])
    rotation_theta = np.array([
        [1, 0, 0],
        [0, math.cos(theta_rad), -math.sin(theta_rad)],
        [0, math.sin(theta_rad), math.cos(theta_rad)],
    ])
    return np.matmul(rotation_theta, np.matmul(rotation_phi, coordinate))

def rotate_face(face, theta_degree, phi_degree):
    temp_face_vertices = face._face.vertices
    temp_face_vertices = np.array(temp_face_vertices).reshape((-1,3))
    new_face_vertices = [rotation_matrix(theta_degree, phi_degree, item) for item in temp_face_vertices]
    new_face_vertices = np.array(new_face_vertices).reshape((1,-1)).tolist()[0]

    temp_face_normals = face._face.normals
    temp_face_normals = np.array(temp_face_normals).reshape((-1,3))
    new_face_normals = [rotation_matrix(theta_degree, phi_degree, item) for item in temp_face_normals]
    new_face_normals = np.array(new_face_normals).reshape((1,-1)).tolist()[0]
    # new_face_normals = [round(item, 3) for item in new_face_normals]

    face.set_vertices(new_face_vertices)
    face.set_normals(new_face_normals)
    face.commit()